In [1]:
import json
import subprocess
from typing import List, Tuple

def get_pairs_to_compare(matrix: List[List[int]]) -> Tuple[List[Tuple[int, int]], float]:
    # Path to the Python script
    python_script_path = "asap_runner.py"

    # Prepare the input for the Python script
    input_data = json.dumps({"matrix": matrix})
    
    # Run the Python script using subprocess
    process = subprocess.Popen(
        ['python', python_script_path],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    # Write the input data to the script
    stdout, stderr = process.communicate(input_data.encode())
    
    if process.returncode != 0:
        raise RuntimeError(f"Error running ASAP script: {stderr.decode()}")

    # Parse the output from the Python script
    result = json.loads(stdout.decode())

    # Extract pairs and max_eig
    pairs = [(pair[0], pair[1]) for pair in result.get("pairs", [])]
    max_eig = result.get("max_eig", 0.0)

    return pairs, max_eig

# Example usage
matrix = [[0] * 100 for _ in range(100)]


pairs, max_eig = get_pairs_to_compare(matrix)
print(f"Pairs: {pairs}")
print(f"Max Eigenvalue: {max_eig}")


Pairs: [(37, 99), (52, 92), (50, 70), (34, 86), (20, 59), (69, 90), (67, 74), (24, 88), (89, 94), (31, 40), (62, 70), (5, 53), (12, 28), (53, 79), (2, 39), (60, 74), (36, 85), (13, 51), (15, 79), (56, 77), (44, 45), (37, 80), (47, 63), (31, 62), (16, 79), (50, 64), (44, 94), (19, 69), (29, 60), (15, 68), (0, 38), (39, 92), (2, 35), (26, 43), (37, 74), (3, 78), (24, 33), (18, 27), (6, 84), (25, 56), (43, 44), (25, 76), (20, 90), (18, 49), (8, 77), (1, 96), (65, 69), (22, 91), (4, 22), (11, 65), (51, 99), (29, 35), (22, 76), (19, 41), (3, 88), (1, 13), (55, 78), (44, 53), (32, 83), (18, 31), (14, 18), (10, 42), (37, 70), (56, 84), (26, 61), (8, 87), (62, 89), (10, 52), (80, 93), (78, 91), (15, 58), (7, 19), (26, 57), (24, 81), (6, 32), (27, 98), (33, 59), (20, 94), (13, 47), (20, 30), (23, 32), (86, 99), (64, 71), (21, 87), (15, 54), (28, 60), (26, 48), (37, 95), (2, 97), (17, 51), (20, 75), (14, 82), (30, 36), (0, 20), (5, 66), (66, 73), (9, 36), (43, 46), (72, 89)]
Max Eigenvalue: 0.99

In [8]:
# Example of running ASAP on CPU
import asap_cpu
import numpy as np
pwc_mat = np.array([[0,20,2,30,1],
                    [1,0,2,3,1],
                    [1,2,0,3,1],
                    [1,2,3,0,1],
                    [5,2,3,1,0]])
N = np.shape(pwc_mat)[0]
asap = asap_cpu.ASAP(N, selective_eig = True)
kl_divs, pairs_to_compare = asap.run_asap(pwc_mat, mst_mode=True)

# Get current score estimation
scores_mean, scores_std = asap.get_scores()



print("Indeces from pwc_mat to compare:")
print(pairs_to_compare)
print("Scores means \n",scores_mean)
print("Scores standard deviaion \n", scores_std)
# print("Expected Information Gain \n", kl_divs)


Indeces from pwc_mat to compare:
[[1 2]
 [0 4]
 [2 3]
 [1 4]]
Scores means 
 [ 0.67833984 -0.510293   -0.3357907  -0.64994346  0.54639733]
Scores standard deviaion 
 [0.21515566 0.25591309 0.296008   0.23823284 0.31112951]


In [ ]:
# Example of running asap_gpu. Note, that due to computational/space optimizations
# selected batch of comparisons might be different from the pure CPU version

import asap_gpu
import numpy as np
pwc_mat = np.array([[0,1,2,3,1],
                    [1,0,2,3,1],
                    [1,2,0,3,1],
                    [1,2,3,0,1],
                    [1,2,3,1,0]])
pairs, scores_mean, scores_std = asap_gpu.ASAP(pwc_mat, mst_mode=True, cuda=True, get_scores = True)

print("Indeces from pwc_mat to compare:")
print(pairs)
print("Scores means \n",scores_mean)
print("Scores standard deviaion \n", scores_std)



In [2]:
# Example of running ASAP-approx on CPU

import asap_cpu
import numpy as np
pwc_mat = np.array([[0,1,2,3,1],
                    [1,0,2,3,1],
                    [1,2,0,3,1],
                    [1,2,3,0,1],
                    [1,2,3,1,0]])
N = np.shape(pwc_mat)[0]
asap = asap_cpu.ASAP(N, selective_eig = True, approx = True)
pairs_to_compare = asap.run_asap(pwc_mat, mst_mode=True)

# Get current score estimation
scores_mean, scores_std = asap.get_scores()

print("Indeces from pwc_mat to compare:")
print(pairs_to_compare)
print("Scores means \n",scores_mean)
print("Scores standard deviaion \n", scores_std)

Indeces from pwc_mat to compare:
[[0 4]
 [0 1]
 [0 2]
 [0 3]]
Scores means 
 [ 0.21775777 -0.03481698 -0.18566486 -0.19490767  0.22731459]
Scores standard deviaion 
 [0.34988997 0.31438954 0.29271232 0.29199316 0.34936609]
